![Qiskit](https://github.com/Qiskit/qiskit-tutorials/raw/115c78962dda85bac29d679063b7d0d0ab1d1ab4/images/qiskit-heading.gif)

# Simulating Noise in Terra, Aqua, and Qiskit Chemistry with Aer

donny@ibm.com <br> _6-Feb-2019_

# Noise in Aer Basics

Qiskit's simulator suite, Aer, contains three types of simulation. 

* The `qasm_simulator` runs individual monte-carlo shots through a quantum circuit to simulate the activity of a real machine. These shots can be performed with perfect statevector evolution, in which case "shot noise" is still present, but can also use a rich set of noise models to simulate evolution in the presence of physical noise. 
    * Note that with no noise model specified, the `qasm_simulator` will only run a single shot under the hood to calculate the final pure statevector, and will simply sample from the distribution described by that statevector for the remaining shots. When a noise model is present, each shot must be run individually because the final statevector for each shot will vary in the presence of physical noise.
* The other types of simulation, `statevector_simulator` and `unitary_simulator` do not allow noise because they do not store the full density matrix in simulation, only a pure statevector.

# Working with Noise in Qiskit

First, install Qiskit Terra, Aer, and the IBMQ package:

In [ ]:
!pip install --no-cache qiskit

Let's install Aqua, Qiskit Chemistry, and Pyscf too while we're at it:

In [ ]:
!pip install qiskit-aqua

In [ ]:
!pip install qiskit-chemistry

In [ ]:
!pip install pyscf

And some imports:

In [ ]:
from qiskit import Aer, IBMQ, execute
from qiskit.providers.aer import noise
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.tools.monitor import job_monitor

# Creating a Noise Model Based on a Real Device

Aer has many noise options that can be tuned, but below we'll use a nifty function in Aer which builds a noise model from the reported noise parameters from a real device, following the instructions in [this notebook](https://github.com/Qiskit/qiskit-tutorials/blob/master/qiskit/aer/device_noise_simulation.ipynb).

In [2]:
IBMQ.load_accounts()

print("Available backends:")
IBMQ.backends()

Available backends:


[<IBMQBackend('ibmq_20_tokyo') from IBMQ(ibm-q-internal, support, default)>,
 <IBMQBackend('ibmq_poughkeepsie') from IBMQ(ibm-q-internal, support, default)>,
 <IBMQBackend('ibmq_qasm_simulator') from IBMQ(ibm-q-internal, support, default)>,
 <IBMQBackend('ibmqx4') from IBMQ()>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ()>,
 <IBMQBackend('ibmq_qasm_simulator') from IBMQ()>]

### Pull device information

In [ ]:
device = IBMQ.get_backend('ibmq_poughkeepsie')
properties = device.properties()
coupling_map = device.configuration().coupling_map

### Construct the Noise Model

In [42]:
from qiskit.providers.aer import noise

In [43]:
noise_model = noise.device.basic_device_noise_model(properties)

# Running a Circuit with Noise in Aer through Terra

Let's build a simple circuit in Terra, and run it with noise.

In [44]:
qr = QuantumRegister(3)
cr = ClassicalRegister(3)
circuit = QuantumCircuit(qr, cr)
circuit.h(qr[0])
circuit.cx(qr[0], qr[1])
circuit.cx(qr[1], qr[2])
circuit.barrier()

circuit.measure(qr, cr)

circuit.draw()

### Running without Noise

In [45]:
simulator = Aer.get_backend('qasm_simulator')

In [46]:
execute(circuit, simulator).result().get_counts()

{'000': 512, '111': 512}

### Running with Our Noise Model

In [47]:
# Get the basis gates for the noise model
basis_gates = noise_model.basis_gates

# Execute noisy simulation and get counts
# Note, this takes a while, ~15 seconds
result_noise = execute(circuit, simulator, 
                       noise_model=noise_model,
                       coupling_map=coupling_map,
                       basis_gates=basis_gates).result()
counts_noise = result_noise.get_counts(circuit)

In [48]:
print(counts_noise)

{'110': 25, '000': 436, '001': 30, '100': 20, '010': 19, '101': 18, '011': 18, '111': 458}


# Running Aqua and Chemistry Calculations with Noise

Here, we're setting up a LiH Ground State Energy calculation:

In [40]:
# Enable aqua logging so we can monitor the VQE run
import logging 
from qiskit_aqua._logging import set_logging_config, build_logging_config, set_aqua_logging
set_aqua_logging(logging.DEBUG)

In [ ]:
from collections import OrderedDict
from qiskit_chemistry import FermionicOperator
from qiskit_chemistry.drivers import PySCFDriver, UnitsType

# Use PySCF, a classical computational chemistry software package, to compute the one- and
# two-body integrals in molecular-orbital basis, necessary to form the Fermionic operator
driver = PySCFDriver(atom='H .0 .0 .0; H .0 .0 0.735', unit=UnitsType.ANGSTROM,
                     charge=0, spin=0, basis='sto3g')
molecule = driver.run()
num_particles = molecule.num_alpha + molecule.num_beta
num_spin_orbitals = molecule.num_orbitals * 2

# Build the qubit operator, which is the input to the VQE algorithm in Aqua
ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
map_type = 'PARITY'
qubitOp = ferOp.mapping(map_type)
qubitOp = qubitOp.two_qubit_reduced_operator(num_particles)
num_qubits = qubitOp.num_qubits

# setup a classical optimizer for VQE
from qiskit_aqua.components.optimizers import SPSA
optimizer = SPSA()

# setup the initial state for the variational form
from qiskit_chemistry.aqua_extensions.components.initial_states import HartreeFock
init_state = HartreeFock(num_qubits, num_spin_orbitals, num_particles)

# setup the variational form for VQE
from qiskit_aqua.components.variational_forms import RYRZ
var_form = RYRZ(num_qubits, initial_state=init_state)

# setup and run VQE
from qiskit_aqua.algorithms import VQE
algorithm = VQE(qubitOp, var_form, optimizer)

### Running VQE with Noise

To run the Aer simulation with noise, we pass the noise model into the QuantumInstance object, which holds all the compilation, backend, and execution information that will be passed to Terra and the backend.

Note, this can take a while to execute.

In [ ]:
from qiskit_aqua import QuantumInstance
quantum_instance = QuantumInstance(backend=simulator, noise_model=noise_model)
result = algorithm.run(quantum_instance)
print(result)